This notebook aims to show how to visualize rotation invariance for the rotation invariant point cloud models. We illustrate this with the cellPACK synthetic dataset. First, lets import some necessary libraries

In [2]:
%load_ext autoreload
%autoreload 2
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = INSERT_YOUR_DEVICE
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-5c1d3311-7294-5551-9e4f-3535560f5f82"
import matplotlib.pyplot as plt
import numpy as np
import torch
import yaml
from hydra.utils import instantiate
from br.features.rotation_invariance import rotation_image_batch_z, rotation_pc_batch_z
from br.models.load_models import load_model_from_path

device = "cuda:0"

Set the CYTODL_CONFIG_PATH environment variable as described in the documentation associated with this repo

In [3]:
# Set paths
# CYTODL_CONFIG_PATH = insert_your_path
CYTODL_CONFIG_PATH = "/allen/aics/modeling/ritvik/projects/latest_clones/benchmarking_representations/configs/"
os.chdir(CYTODL_CONFIG_PATH)
os.chdir("..")
# save_path = insert_save_path
save_path = './test_cellpack_recons/'
results_path = CYTODL_CONFIG_PATH + '/results/'

Load a batch of the cellPACK point cloud dataset

In [12]:
# Load data yaml and test batch
cellpack_data = CYTODL_CONFIG_PATH + "data/cellpack/pc.yaml"
with open(cellpack_data) as stream:
    cellpack_data = yaml.safe_load(stream)
data = instantiate(cellpack_data)
batches = []
for batch in data.train_dataloader():
    print(batch['orig_CellId'][0])
    if batch['orig_CellId'][0] == '7abfecf1-44db-468a-b799-4959a23cfb0d':
        batches.append(batch)
batch = batches[0]

get_packings: 100%|██████████| 1062/1062 [00:05<00:00, 177.47it/s]


a36cb830-d263-4d36-b1c9-f44996deb009
f62f2072-a7fe-47ba-bbb3-63ca0f3254e5
54418656-e492-4e69-bbe7-41b35cfb8994
1332bc99-9f33-4b2a-a87a-0c0ad57e5d7b
4a01ee10-4b34-4086-9bbc-6219a6db23fe
6a7f2c1a-7ce3-423e-9cc6-90c467be3165
33a81e81-255e-4331-b611-2483cb2a9a9e
0d73bbf2-7d44-45da-8ff2-177c217beea7
f63fbba3-7ac1-45c8-8cb7-3ab12fddad2a
d7508cd0-5286-4108-9536-d50eb74206fb
d09b439f-5632-426b-8aa2-cea0cba3439a
9c1ff213-4e9e-4b73-a942-3baf9d37a50f
64869729-ef0e-469a-b287-c6237a4ba811
0f90a504-217d-4312-84d6-caa082b3fbb3
859cd04e-7890-407f-81aa-4a71019d0468
88edf2b3-931c-4c38-a092-0b86e08716db
829d9af2-23e9-4422-b109-ba89f9911cb1
d8a208d0-b569-4c16-9cac-1d1c53f9265e
03eb83aa-d025-438a-b346-e741056c47b6
0b228acc-d475-432f-9b84-0cb09c429d95
6fe102cc-d911-447b-a290-d5126d65a8bb
d531a2f2-1ef2-418f-a834-011be59cd195
8ba97b46-9da5-4b55-83cf-460268b2f3cc
e2c7f55f-00be-4ca9-9d46-9f0eeae6dbfa
5d8fa2b6-9e9b-4517-a805-b074b50de03c
4ebe8f0b-aebe-495a-a8b9-0bb6d2297c2a
3cb5aea3-cfa5-48ef-8c6b-882dc62d0986
2

IndexError: list index out of range

In [11]:
for batch in data.train_dataloader():
    print(batch['orig_CellId'][0])
    if batch['orig_CellId'][0] == '7abfecf1-44db-468a-b799-4959a23cfb0d':
        print('yes')
        batches.append(batch)

get_packings: 100%|██████████| 1062/1062 [00:06<00:00, 168.29it/s]


a36cb830-d263-4d36-b1c9-f44996deb009
f62f2072-a7fe-47ba-bbb3-63ca0f3254e5
54418656-e492-4e69-bbe7-41b35cfb8994
1332bc99-9f33-4b2a-a87a-0c0ad57e5d7b
4a01ee10-4b34-4086-9bbc-6219a6db23fe
6a7f2c1a-7ce3-423e-9cc6-90c467be3165
33a81e81-255e-4331-b611-2483cb2a9a9e
0d73bbf2-7d44-45da-8ff2-177c217beea7
f63fbba3-7ac1-45c8-8cb7-3ab12fddad2a
d7508cd0-5286-4108-9536-d50eb74206fb
d09b439f-5632-426b-8aa2-cea0cba3439a
9c1ff213-4e9e-4b73-a942-3baf9d37a50f
64869729-ef0e-469a-b287-c6237a4ba811
0f90a504-217d-4312-84d6-caa082b3fbb3
859cd04e-7890-407f-81aa-4a71019d0468
88edf2b3-931c-4c38-a092-0b86e08716db
829d9af2-23e9-4422-b109-ba89f9911cb1
d8a208d0-b569-4c16-9cac-1d1c53f9265e
03eb83aa-d025-438a-b346-e741056c47b6
0b228acc-d475-432f-9b84-0cb09c429d95
6fe102cc-d911-447b-a290-d5126d65a8bb
d531a2f2-1ef2-418f-a834-011be59cd195
8ba97b46-9da5-4b55-83cf-460268b2f3cc
e2c7f55f-00be-4ca9-9d46-9f0eeae6dbfa
5d8fa2b6-9e9b-4517-a805-b074b50de03c
4ebe8f0b-aebe-495a-a8b9-0bb6d2297c2a
3cb5aea3-cfa5-48ef-8c6b-882dc62d0986
2

In [7]:
batch['orig_CellId']

['4157e41a-f5f1-456c-b2c0-13bc41cdb73a',
 '4157e41a-f5f1-456c-b2c0-13bc41cdb73a',
 '4157e41a-f5f1-456c-b2c0-13bc41cdb73a',
 '4157e41a-f5f1-456c-b2c0-13bc41cdb73a',
 '4157e41a-f5f1-456c-b2c0-13bc41cdb73a',
 '4157e41a-f5f1-456c-b2c0-13bc41cdb73a']

Save examples of raw data for each of the 6 packing rules

In [20]:
from pathlib import Path

this_save_path = Path(save_path) / Path("panel_a")
this_save_path.mkdir(parents=True, exist_ok=True)

all_arr = []
for i in range(6):
    np_arr = batch["pcloud"][i].numpy()
    new_array = np.zeros(np_arr.shape)
    z = np_arr[:, 0]
    new_array[:, 0] = np_arr[:, 2]
    new_array[:, 1] = z
    new_array[:, 2] = np_arr[:, 1]
    all_arr.append(new_array)
    np.save(this_save_path / Path(f"{i}.npy"), new_array)

IndexError: index 2 is out of bounds for dimension 0 with size 2

Make utility functions for computing and saving reconstructions across four 90 degree rotations 

In [ ]:
# utility function for plotting
def plot_pc(this_p, axes, max_size, color="gray", x_ind=2, y_ind=1):
    axes.scatter(this_p[:, x_ind], this_p[:, y_ind], c=color, s=1)
    axes.spines["top"].set_visible(False)
    axes.spines["right"].set_visible(False)
    axes.spines["bottom"].set_visible(False)
    axes.spines["left"].set_visible(False)
    axes.set_aspect("equal", adjustable="box")
    axes.set_ylim([-max_size, max_size])
    axes.set_xlim([-max_size, max_size])
    axes.set_yticks([])
    axes.set_xticks([])

def save_recons_across_rotations(save_path, this_name, this_key, model, batch):
    this_save_path = Path(save_path) / Path(f"Recons_{this_name}")
    this_save_path.mkdir(parents=True, exist_ok=True)

    max_z = {0: 20, 1: 20, 2: 20, 3: 1, 4: 20, 5: 20}
    max_size = 10

    all_thetas = [
        0,
        1 * 90,
        2 * 90,
        3 * 90,
    ]


    all_xhat = []
    all_canon = []
    all_input = []
    with torch.no_grad():
        for jl, theta in enumerate(all_thetas):
            this_input_rot = rotation_pc_batch_z(
                batch,
                theta,
            )
            batch_input = {this_key: torch.tensor(this_input_rot).to(device).float()}
            z, z_params = model.get_embeddings(batch_input, inference=True)
            xhat = model.decode_embeddings(z_params, batch_input, decode=True, return_canonical=True)
            all_input.append(this_input_rot)
            for ind in range(6):
                this_p = this_input_rot[ind]
                this_max_z = max_z[ind]
                this_p = this_p[np.where(this_p[:, 0] < this_max_z)[0]]
                this_p = this_p[np.where(this_p[:, 0] > -this_max_z)[0]]
                fig, ax = plt.subplots(1, 1, figsize=(5, 5))
                plot_pc(this_p, ax, max_size, "black")
                fig.savefig(this_save_path / f"input_{ind}_theta_{theta}.png")

            if ("canonical" in xhat.keys()) and ("Rotation" in this_name):
                this_canon = xhat["canonical"].detach().cpu().numpy()
                all_canon.append(this_canon)
                for ind in range(6):
                    this_p = this_canon[ind]
                    this_max_z = max_z[ind]
                    this_p = this_p[np.where(this_p[:, 1] < this_max_z)[0]]
                    this_p = this_p[np.where(this_p[:, 1] > -this_max_z)[0]]
                    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
                    plot_pc(this_p, ax, max_size, "black", x_ind=0, y_ind=2)
                    fig.savefig(this_save_path / f"canon_{ind}_theta_{theta}.png")
            this_recon = xhat[this_key].detach().cpu().numpy()
            all_xhat.append(this_recon)
            for ind in range(6):
                this_p = this_recon[ind]
                this_max_z = max_z[ind]
                this_p = this_p[np.where(this_p[:, 0] < this_max_z)[0]]
                this_p = this_p[np.where(this_p[:, 0] > -this_max_z)[0]]
                fig, ax = plt.subplots(1, 1, figsize=(5, 5))
                plot_pc(this_p, ax, max_size, "black")
                fig.savefig(this_save_path / f"recon_{ind}_theta_{theta}.png")

Load the trained models

In [ ]:
models, names, sizes, model_manifest, x_labels, latent_dims = load_model_from_path("cellpack", results_path)

Move batch elements to gpu

In [ ]:
for key in batch.keys():
    if key not in [
        "split",
        "bf_meta_dict",
        "egfp_meta_dict",
        "filenames",
        "image_meta_dict",
        "cell_id",
    ]:
        if not isinstance(batch[key], list):
            batch[key] = batch[key].to(device)

Save reconstructions across rotations for the classical point cloud model

In [ ]:
# Lets load the classical point cloud model
ind = names.index('Classical_pointcloud')
model = models[ind]
this_name = names[ind]
this_key = x_labels[ind]
save_recons_across_rotations(save_path, this_name, this_key, model, batch)

Save reconstructions across rotations for the rotation invariant point cloud model

In [ ]:
# Lets load the rotation invariant point cloud model
ind = names.index('Rotation_invariant_pointcloud')
model = models[ind]
this_name = names[ind]
this_key = x_labels[ind]
save_recons_across_rotations(save_path, this_name, this_key, model, batch)